# Supplier Sourcing

We will now formulate and solve a supplier sourcing problem.

New Bedford Steel (NBS) procures coking coal to produce steel. For next year’s
production, NBS has solicited the following bids from eight different coal mines:

$$
\begin{array}{c | cccccccc}
\hline
\hline
 & Ashley & Bedfort & Consol &Dunby &Earlam &Florence &Gaston &Hopt \\
\hline
Price (\$/mt) & 49,500 & 50,000 & 61,000 &63,500 &66,500 &71,000 &72,500 &80,000  \\
Union? & yes & yes & no & yes &no &yes &no &no \\
Transport & rail & truck & rail &truck &truck &truck &rail &rail \\
Volatility (percentage) & 15 & 16 & 18 & 20 &21 &22 &23 &25 \\
Capacity (mt/yr) &300 &600 &510 &655 &575 &680 &450 &490 \\
\hline
\hline
\end{array}
$$

NBS wants to procure 1,225mt of coking coal with an average volatility of at least 19%. To avert adverse labour relations, at least 50% of the coal should come from union mines. Moreover, at most 650mt (720mt) can be transported via rail (trucks).

## Questions

*   How much should NBS procure from each mine so as to minimize total costs?
*   What are the total costs? What are the total purchase cost from each supplier?

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_csv("suppliers.csv", index_col=0)

In [ ]:
cap_chart = px.bar(
    x=df.index, y=df.Capacity,
    labels={"x":"Supplier","y":"Capacity"})

In [ ]:
demand = 1225

In [ ]:
avg_vola = 0.19

In [ ]:
max_rail = 650

In [ ]:
max_trucks = 750

In [ ]:
max_union = 0.5

## Model Formulation

The problem can be formulated as a linear program. The objective is to minimize the total cost of allocating volums to suppliers subject to a number of constraints.
$$
\begin{align}
&\min \quad && \sum_i \text{price}_i \text{volume}_i && \text{minimize total cost}\\
&s.t. && \sum_i \text{volume}_i = 1225 && \text{satisfy demand}\\
& &&\sum_i \text{volatility}_i \text{volume}_i \geq 0.19 \cdot 1225 && \text{meet volatility target}\\
& && \sum_i \text{union}_i \text{volume}_i \geq 0.5 \cdot 1225 && \text{meet union target} \\
& && \sum_i \text{rail}_i \text{volume}_i \leq 650 && \text{respect rail transport limit} \\
& && \sum_i (1-\text{rail}_i) \text{volume}_i \leq 750 && \text{respect truck transport limit} \\
& && 0 \leq \text{volume}_i \leq \text{capacity}_i\ \forall \ i=1,\dots,I  && \text{respect supplier capacities}
\end{align}
$$

In [ ]:
import cvxpy as cp
volumes = cp.Variable(len(df))
objective = cp.Minimize(df.Price.values@volumes)
constraints = [cp.sum(volumes) == demand]
constraints.append(df.Volatility.values@volumes >= (avg_vola)*cp.sum(volumes))
constraints.append(df.Rail.values@volumes >= (1/2)*cp.sum(volumes))
constraints.append(df.Rail.values@volumes <= max_rail)
constraints.append((1-df.Rail.values)@volumes <= max_trucks)
constraints.append(volumes[:] <= df.Capacity.values[:])
constraints.append(volumes >= 0)
prob = cp.Problem(objective, constraints)
min_cost = prob.solve(solver=cp.ECOS)
print("Supplier selection problem solution status: %s"%prob.status)

In [ ]:
vol_chart = px.bar(x=df.index, y=volumes.value,
    labels={"x":"Supplier","y":"Purchase Volume"})

In [ ]:
cost_chart = px.bar(x=df.index, y=(volumes.value*df.Price.values),
    labels={"x":"Supplier","y":"Total Cost"},
    color_discrete_sequence=["red"])